In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

In [12]:
# Opening json file
import os
import sys
import pandas as pd

file = open("/Users/naeeramin/Documents/UTS_3rd_semester/ilab2/github_repo/ARDC-Project/results2022.json", "r")

In [13]:
data = json.loads(file.read())

In [30]:
totalEmployees = 0 
successfulGoogleQuery = 0
correctProfileScraped = 0
workForGov = 0
for employees in data:
    totalEmployees += 1
    if employees["SuccessfulGoogleQuery"]:
        successfulGoogleQuery += 1
        for profiles in employees["ProfilesScraped"]:
            if profiles["CorrectName"] and profiles["CorrectCompany"]:
                correctProfileScraped += 1
                if profiles["WorkedForGovernment"]:
                    workForGov += 1

print(totalEmployees)
print(successfulGoogleQuery)
print(correctProfileScraped)
print(workForGov)

print("Percentage of successful Google Queries: " + str((successfulGoogleQuery/totalEmployees)*100))
print("Percentage of Correct Profiles Scraped: " + str((correctProfileScraped/successfulGoogleQuery)*100))
print("Percentage of Employees with Government Experience from correct profiles scraped: " + str((workForGov/correctProfileScraped)*100))

        

251
237
116
88
Percentage of successful Google Queries: 94.42231075697211
Percentage of Correct Profiles Scraped: 48.9451476793249
Percentage of Employees with Government Experience from correct profiles scraped: 75.86206896551724
